<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-import-and-preparation" data-toc-modified-id="Data-import-and-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data import and preparation</a></span><ul class="toc-item"><li><span><a href="#Calculating-key-KPIs" data-toc-modified-id="Calculating-key-KPIs-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Calculating key KPIs</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import random as random

# plotly standard imports
from datetime import datetime
from datetime import timedelta  

In [4]:
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.figure_factory as ff


import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py

# Data import and preparation

In [5]:
data_input=pd.read_excel("raw_data.xlsx", sep=";")

In [6]:
dataset_downland_date=data_input.data_ts.max()
dataset_downland_date

Timestamp('2019-08-23 00:00:00')

In [23]:
df=df[df.payment_date<=dataset_downland_date]
df.shape

(37311, 19)

In [24]:
df.head(10)

,loan_ID,client_ID,sex,age,post_code,cash_payment,initial_loan_value,installment_count,payout_date,first_installment_date,installment,period,payment_value,payment_date,cumulated_payments,expected_payment_date,last_payment_period,last_payment_date,max_cumulated_payments
46170,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,1,244.68,2017-07-19,244.68,2017-07-20,13,2018-07-27,3194.68
46171,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,2,245.00,2017-08-22,489.68,2017-08-20,13,2018-07-27,3194.68
46172,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,3,245.00,2017-09-18,734.68,2017-09-20,13,2018-07-27,3194.68
46173,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,4,245.00,2017-10-20,979.68,2017-10-20,13,2018-07-27,3194.68
46174,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,5,245.00,2017-11-17,1224.68,2017-11-20,13,2018-07-27,3194.68
46175,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,6,245.00,2017-12-20,1469.68,2017-12-20,13,2018-07-27,3194.68
46176,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,7,245.00,2018-01-22,1714.68,2018-01-20,13,2018-07-27,3194.68
46177,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,8,245.00,2018-02-20,1959.68,2018-02-20,13,2018-07-27,3194.68
46178,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,9,245.00,2018-03-20,2204.68,2018-03-20,13,2018-07-27,3194.68
46179,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,10,245.00,2018-04-19,2449.68,2018-04-20,13,2018-07-27,3194.68


## Calculating key KPIs

In [25]:
df["payment_delay"]=df.payment_date-df.expected_payment_date
df.payment_delay=df.payment_delay.apply(lambda x: x.days)
df["month"]=df.expected_payment_date.apply(lambda x: x.month)
df["payment_completeness"]=df.payment_value/df.installment
df['paid_ratio']=df.cumulated_payments/df.initial_loan_value


In [26]:
df["year"]=df.payment_date.apply(lambda x:x.year)

In [27]:
df["is_male"]=np.where(df.sex=="Male",1,0)
df["is_christmass_season"]=df.month.apply(lambda x: x in(1,2,12))
df.is_christmass_season=np.where(df.is_christmass_season,1,0)

In [28]:
threshold=0.2
df["is_default"]=np.where(df.payment_completeness<threshold,1,0)
df.payment_completeness=np.where(df.payment_completeness>2,2, df.payment_completeness)

In [29]:
df.reset_index(inplace=True)
import datetime
df["month_year"]=np.NaN
for i in range(0,len(df)):
   
    df["month_year"][i]=datetime.date(year=df.payment_date[i].year,month=df.payment_date[i].month,day=1)
    

In [30]:
df["date_bucket"] = pd.cut(df.payment_date,25)
df["date_bucket"]=df["date_bucket"].apply(lambda x:x.right)


In [31]:
df.drop(columns=["index"],inplace=True)

In [32]:
df.payment_delay=np.where(df.payment_delay>60,60,df.payment_delay)
df.payment_delay=np.where(df.payment_delay<-60,-60,df.payment_delay)